In [1]:
import numpy as np
list_rank = np.array([6,7,8,11,12,15])
np.random.seed(5)
np.random.shuffle(list_rank)
list_rank

array([15,  8, 12,  7,  6, 11])

In [2]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
class Dataset(Dataset):
    def __init__(self,train_x,train_y):
        self.train_x=train_x
        self.train_y=train_y
        self.data_len = len(train_x)

    def __getitem__(self, index):

        train_x=self.train_x[index]
        train_y=self.train_y[index]
        return (train_x, train_y)
    def __len__(self):
        return self.data_len
    
class dataset_dataloader:
    
    def __init__ (self,is_whole,is_cross,method,rank,cross_number_for_test=1,batchsize=24,numworkers=2,seed=1):
        import numpy as np
        if   rank =="a":
            list_rank=[6,7,8,11,12,15]
            self.rank=list_rank
        elif rank =="d":
            list_rank=[15,12,11,8,7,6]
            self.rank=list_rank
        elif rank =="now":
            list_rank=[11,12,15,6,7,8]
            self.rank=list_rank
        elif rank =="random":
            list_rank = np.array([6,7,8,11,12,15])
            np.random.seed(seed)
            np.random.shuffle(list_rank)
            self.rank=list_rank
            
        if is_cross==False:
            import pandas as pd
            
            from PIL import Image
            from pyts.visualization import plot_gasf
            import matplotlib.pyplot as plt
            import torch
            from torch.autograd import Variable

            if is_whole==True:
                whole="_all"
            else:
                whole=""

            normal   = pd.read_json("normal_train"+whole+".json")
            abnormal = pd.read_json("abnormal_train"+whole+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            def pil_loader(path):
                with open(path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('L')
            lst = list()
            count=0
            lst2=list()
            y=list()

            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="wafer_img"+whole+"/"+method+"/train/abnormal/"
                else:
                    path="wafer_img"+whole+"/"+method+"/train/normal/"

                list_im = [
                    path+str(i)+'_'+str(list_rank[0])+'.png', 
                    path+str(i)+'_'+str(list_rank[1])+'.png', 
                    path+str(i)+'_'+str(list_rank[2])+'.png', 
                    path+str(i)+'_'+str(list_rank[3])+'.png', 
                    path+str(i)+'_'+str(list_rank[4])+'.png', 
                    path+str(i)+'_'+str(list_rank[5])+'.png', 
                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)

                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     count+=1
            #     if count==1:
            #         break
            train_x = torch.from_numpy(np.array(lst2)).float()
            train_y = torch.LongTensor(np.array(y))

            train_data = Dataset(train_x,train_y)
            train_loader = DataLoader(dataset=train_data,
                                      batch_size=24,
                                      shuffle=True,
                                      num_workers=2)
            self.train_x=train_x
            self.train_y=train_y
            self.train_loader=train_loader

            #test data set
            normal   = pd.read_json("normal_validation"+whole+".json")
            abnormal = pd.read_json("abnormal_validation"+whole+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            # print(x)
            lst = list()
            count=0
            lst2=list()
            y=list()
            path=""
            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="wafer_img"+whole+"/"+method+"/validation/abnormal/"
                else:
                    path="wafer_img"+whole+"/"+method+"/validation/normal/"

                list_im = [
                    path+str(i)+'_'+str(list_rank[0])+'.png', 
                    path+str(i)+'_'+str(list_rank[1])+'.png', 
                    path+str(i)+'_'+str(list_rank[2])+'.png', 
                    path+str(i)+'_'+str(list_rank[3])+'.png', 
                    path+str(i)+'_'+str(list_rank[4])+'.png', 
                    path+str(i)+'_'+str(list_rank[5])+'.png', 
                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)
                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     if count==1:
            #         break
            test_x = Variable(torch.from_numpy(np.array(lst2)).float()).cuda()
            test_y = torch.LongTensor(np.array(y)).cuda()
            self.test_x=test_x
            self.test_y=test_y        

            print("done")
            
        else:
            import pandas as pd
            import numpy as np
            from PIL import Image
            from pyts.visualization import plot_gasf
            import matplotlib.pyplot as plt
            import torch
            from torch.autograd import Variable

            if is_whole==True:
                whole="_all"
            else:
                whole=""
            cross=["1","2","3","4","5"]
            cross.remove(str(cross_number_for_test))
            x = [  
                pd.read_json("normal"+whole+"_cross_"+cross[0]+".json"),
                pd.read_json("normal"+whole+"_cross_"+cross[1]+".json"),
                pd.read_json("normal"+whole+"_cross_"+cross[2]+".json"),
                pd.read_json("normal"+whole+"_cross_"+cross[3]+".json"),
                pd.read_json("abnormal"+whole+"_cross_"+cross[0]+".json"),
                pd.read_json("abnormal"+whole+"_cross_"+cross[1]+".json"),
                pd.read_json("abnormal"+whole+"_cross_"+cross[2]+".json"),
                pd.read_json("abnormal"+whole+"_cross_"+cross[3]+".json")
            ]
            x = pd.concat(x)
            x =  x.sample(frac=1)


            def pil_loader(path):
                with open(path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('L')
            lst = list()
            count=0
            lst2=list()
            y=list()

            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="wafer_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/abnormal/"
                else:
                    path="wafer_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/normal/"

                list_im = [
                    path+str(i)+'_'+str(list_rank[0])+'.png', 
                    path+str(i)+'_'+str(list_rank[1])+'.png', 
                    path+str(i)+'_'+str(list_rank[2])+'.png', 
                    path+str(i)+'_'+str(list_rank[3])+'.png', 
                    path+str(i)+'_'+str(list_rank[4])+'.png', 
                    path+str(i)+'_'+str(list_rank[5])+'.png', 
                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)

                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     count+=1
            #     if count==1:
            #         break
            train_x = torch.from_numpy(np.array(lst2)).float()
            train_y = torch.LongTensor(np.array(y))

            train_data = Dataset(train_x,train_y)
            train_loader = DataLoader(dataset=train_data,
                                      batch_size=24,
                                      shuffle=True,
                                      num_workers=2)
            self.train_x=train_x
            self.train_y=train_y
            self.train_loader=train_loader

            #test data set
            normal   = pd.read_json("normal"+whole+"_cross_"+str(cross_number_for_test)+".json")
            abnormal = pd.read_json("abnormal"+whole+"_cross_"+str(cross_number_for_test)+".json")
            x=normal.append(abnormal)
            x=x.sample(frac=1)

            # print(x)
            lst = list()
            count=0
            lst2=list()
            y=list()
            path=""
            for i in x.batch.unique():
                df=x[x.batch==i]
                lst = list()
                if np.all(df.status=="abnormal"):
                    path="wafer_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/abnormal/"
                else:
                    path="wafer_img_cross"+whole+"/"+method+"/"+str(df.iloc[0].cross)+"/normal/"

                list_im = [
                    path+str(i)+'_'+str(list_rank[0])+'.png', 
                    path+str(i)+'_'+str(list_rank[1])+'.png', 
                    path+str(i)+'_'+str(list_rank[2])+'.png', 
                    path+str(i)+'_'+str(list_rank[3])+'.png', 
                    path+str(i)+'_'+str(list_rank[4])+'.png', 
                    path+str(i)+'_'+str(list_rank[5])+'.png', 
                ]
                imgs = [ pil_loader(i) for i in list_im ]
                min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
                imgs_comb = Image.fromarray( imgs_comb)
                lst.append(np.array(imgs_comb)/255)
                arr = np.array(lst)
                if df.iloc[0].status=="normal":
                    y.append(0)
                else:
                    y.append(1)

                lst2.append(arr)    
            #     if count==1:
            #         break
            test_x = Variable(torch.from_numpy(np.array(lst2)).float()).cuda()
            test_y = torch.LongTensor(np.array(y)).cuda()
            self.test_x=test_x
            self.test_y=test_y        

            print("done")
            
        
print("Function setting done")


Function setting done


In [3]:
cross=dataset_dataloader(True,True,"mtf","d")

done


In [4]:
### import torch.nn as nn
import torch
from torch.autograd import Variable
accuracy_matrix =[]
import dataset
import dataset_six

whole=[True]
method=["a","d","random"]

F = open('rank.txt', 'w')  
for l in method:
    for m in whole:
        for k in range(20):
            accuracy_matrix=[]
            for i in range(5):
                min_error=100
                min_lose=100
                count=0
                number=1
                cross=dataset_dataloader(m,True,"mtf",l,i+1)
                import torch.nn as nn

                EPOCH = 50              # train the training data n times, to save time, we just train 1 epoch
                LR = 0.0023

                class CNN(nn.Module):
                    def __init__(self):
                        super(CNN, self).__init__()
                        self.conv1 =nn.Sequential( #卷基層   
                            nn.Conv2d(
                                in_channels  = 1  , # 圖片是有幾層的 若 RGB三層 灰階 1層
                                out_channels = 3, # 同時16個filter 進行掃描 會提取16個特徵 代表下一層高度為16
                                kernel_size  = 5  , # 一次畫出來的框 畫幾格 ex 5*5
                                stride       = 1  , # 每一個框框跳幾格
                                padding      = 2, # 在 5x5逐步掃描後會有少的格子 因此2代表在外面為兩圈的0
                                                    # if stride =1 padding = (kernel_size-1)/2
                            ),  # 過濾器 卷基層 蒐集圖片訊息 三維的空間  
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2), #找出更重要的特徵  像是在2x2中 找到 最大的值
                        )
                        self.conv2 = nn.Sequential(
                            nn.Conv2d(3,5,5,1,2), # 輸入的是上一層的16 把輸出層在變大因此是32 其他不變
                            nn.ReLU(),
                            nn.MaxPool2d(2)
                        )
            #             self.conv3 = nn.Sequential(
            #                 nn.Conv2d(10,20,3,1,1), # 輸入的是上一層的16 把輸出層在變大因此是32 其他不變
            #                 nn.ReLU(),
            # #                 nn.MaxPool2d(2)
            #             )

                        self.out = nn.Linear(5*192*32,2)
                    def forward(self,x):

                        x=self.conv1(x)
                        x=self.conv2(x) #(batch.32.7.7)
            #             x=self.conv3(x)
                        x=x.view(x.size(0),-1) #將三維轉二維 (batch , 32*7*7)
                        output=self.out(x)
                        return output,x

                cnn=CNN()
                cnn.cuda()
            #     print(cnn)

                optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
                loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

                # following function (plot_with_labels) is for visualization, can be ignored if not interested

                # training and testing


                for epoch in range(EPOCH):
                    for step, (x, y) in enumerate(cross.train_loader):   # gives batch data, normalize x when iterate train_loader
                        b_x = Variable(x).cuda()   # batch x
                        b_y = Variable(y).cuda()   # batch y

                        output = cnn(b_x)[0]             # cnn output
                        loss = loss_func(output, b_y)   # cross entropy loss
                        optimizer.zero_grad()           # clear gradients for this training step
                        loss.backward()                 # backpropagation, compute gradients
                        optimizer.step()                # apply gradients
                        test_output, last_layer = cnn(cross.test_x)
                        pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()
                        accuracy = sum(pred_y == cross.test_y) / float(cross.test_y.size(0))
                        error  = (1-accuracy)*100
                        if error<min_error:
                            min_error=error
                            min_lose=100
                            count=0
                        else:
                            count+=1
                            print(".",end="")

                        if error == min_error:
                            if loss.data[0]<min_lose:
                                min_lose=loss.data[0]
                                torch.save(cnn,'cnn_save/cnn'+str(number)+'.pkl')
                                number+=1
                #                 print('save!!')
                #                 print('\ncount: ',count,'Epoch: ', epoch, '| train loss: %.15f' % loss.data[0], '| min validation error rate: %.20f' % min_error)
                        if count>=500:
                            break
                    if count>=500:
                        break

                # print 10 predictions from test data
                # test_output, _ = cnn(test_x)
                # pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()

                # print(pred_y, 'prediction number')
                # print(test_y, 'real number')
                accuracy_matrix.append(min_error)
                print('\n number: ',number-1 ,min_error)
            import IPython.display as ipd
            import numpy
            sr = 22050 # sample rate
            T = 0.3    # seconds
            t = numpy.linspace(0, T, int(T*sr), endpoint=False) # time variable
            x = 0.5*numpy.sin(2*numpy.pi*490*t)                # pure sine wave at 440 284Hz

            print(accuracy_matrix)
            sum(accuracy_matrix) / float(len(accuracy_matrix))
            F.write(str(l)+","+str(m)+","+str(k)+",")
            for i in accuracy_matrix:  
                F.write(str(i) + ",")
            F.write(str(sum(accuracy_matrix) / float(len(accuracy_matrix)))+"\n")
F.close()  


Function setting done
Function setting done
done


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  32 1.6806722689075682
done
......................................................................................................................................................................................................................................................................................................................................

................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  33 0.8368200836820106
done
..................................................................................................................................................................................................................................................

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  36 0.0
done
................................................................................................................................................................................................................................................................................................................................................

...

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  31 0.0
done
................................................................................................................................................................................................................................................................................................................................................................................................................................................

......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  31 1.538461538461533
done
.............................................................................................................................................................................................................................................................................................................................................................................................................................................

.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  27 1.5151515151515138
done
.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  25 0.0
done
................................................................................................................................................................................................................................................................................................................................................................................................................................................

.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
 number:  18 1.538461538461533
done
..............................................................................................................................................................................................................................................................................................................................................................................................................................................